In [2]:
! pip install kaggle -q
! pip install nltk

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gregoryeritsyan/im2latex-230k")

print("Path to dataset files:", path)

100%|██████████| 969M/969M [00:12<00:00, 81.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gregoryeritsyan/im2latex-230k/versions/1


In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"michaeld0lgov","key":"622f6f4344d42d402cc39ce81055a8e9"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [6]:
!ls -ltr ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'gregoryeritsyan/im2latex-230k'

total 4
-rw-r--r-- 1 root root 69 Dec 15 22:46 kaggle.json
Dataset URL: https://www.kaggle.com/datasets/gregoryeritsyan/im2latex-230k
License(s): CC0-1.0
100% 966M/969M [00:12<00:00, 246MB/s]
100% 969M/969M [00:12<00:00, 82.2MB/s]


In [ ]:
! ls /content
! unzip im2latex-230k.zip

Streaming output truncated to the last 5000 lines.
  inflating: PRINTED_TEX_230k/generated_png_images/fab3f77c62edeec.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab4302dbb03db9.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab48e922e46907.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab498372f1840e.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab4b3feed2cd43.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab4f497733549d.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab50908c1b149b.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab536c5b2dc11c.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab542e4fb99a9f.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab62f37881431e.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab63e62813bf70.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab648fb9c3a15f.png  
  inflating: PRINTED_TEX_230k/generated_png_images/fab7d25011ce9b0.png  


In [ ]:
! ls /content/PRINTED_TEX_230k/
! ls -ltr /content/PRINTED_TEX_230k/39fedf5461c6f2b.png

230k.json		      final_png_formulas.txt
corresponding_png_images.txt  generated_png_images
ls: cannot access '/content/PRINTED_TEX_230k/39fedf5461c6f2b.png': No such file or directory


In [ ]:
!find /content/PRINTED_TEX_230k/generated_png_images -name "39fedf5461c6f2b.png"


/content/PRINTED_TEX_230k/generated_png_images/39fedf5461c6f2b.png


In [ ]:
! ls /content/PRINTED_TEX_230k/
! ls -ltr /content/PRINTED_TEX_230k/39fedf5461c6f2b.png

230k.json		      final_png_formulas.txt
corresponding_png_images.txt  generated_png_images
ls: cannot access '/content/PRINTED_TEX_230k/39fedf5461c6f2b.png': No such file or directory


In [ ]:
! pip install uv -q
! pip install opencv-python -q
! pip install pillow -q
! pip install matplotlib -q
! uv pip install texteller -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 81.6 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize

def marzal_vidal(reference, prediction):
    reference = str(reference)
    prediction = str(prediction)

    n, m = len(reference), len(prediction)
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = 0 if reference[i - 1] == prediction[j - 1] else 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,
                dp[i][j - 1] + 1,
                dp[i - 1][j - 1] + cost
            )

    distance = dp[n][m]
    path_len = max(n, m)

    normalized = distance / path_len if path_len != 0 else 0.0

    return normalized

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def bleu_score_str(reference, prediction):
  nltk.download('punkt_tab', quiet=True)
  if isinstance(reference, str):
    refs = [reference]
  else:
    refs = list(reference)
  ref_tokens = [word_tokenize(r) for r in refs]
  hyp_tokens = word_tokenize(prediction)
  if len(hyp_tokens) == 0 or any(len(r) == 0 for r in ref_tokens):
    return 0.0
  sf = SmoothingFunction()
  score = sentence_bleu(ref_tokens, hyp_tokens, smoothing_function=sf.method1)
  return score


In [ ]:
import torch
import numpy as np
from transformers import GPT2TokenizerFast, GPT2Model
import re
import math


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_models_and_tokenizer():
    global tokenizer, gpt2_model, embedding_layer, positional_embedding, new_embeddings

    tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
    gpt2_model = GPT2Model.from_pretrained('gpt2')
    embedding_layer = gpt2_model.wte.to(device)
    positional_embedding = gpt2_model.wpe.to(device)

    try:
        if device == 'cuda':
            new_embeddings_state = torch.load('new_embeddings.pth')
        else:
            new_embeddings_state = torch.load('new_embeddings.pth', map_location=torch.device('cpu'))

        new_vocab_size, embedding_dim = new_embeddings_state['weight'].shape
        new_embeddings = torch.nn.Embedding(new_vocab_size, embedding_dim).to(device)
        new_embeddings.load_state_dict(new_embeddings_state)
    except FileNotFoundError:
        print("Warning: new_embeddings.pth not found. Using default embeddings.")
        new_embeddings = None


def spacing(text):
    result = []
    for i, char in enumerate(text):
        if char == "\\":
            if i == 0 or text[i-1] != " ":
                result.append(" \\")
            else:
                result.append("\\")
        else:
            result.append(char)
    return ''.join(result)


def get_token_embeddings(sentence):
    sentence = spacing(sentence)
    tokens = tokenizer.encode(sentence, truncation=True, max_length=512)
    decoded_tokens = [tokenizer.decode([token]) for token in tokens]
    print(f"Decoded tokens: {decoded_tokens}")

    token_ids = torch.tensor(tokens).unsqueeze(0).to(device)
    positions = torch.arange(0, token_ids.size(1)).unsqueeze(0).to(device)

    if new_embeddings is not None:
        token_embeddings = torch.cat([embedding_layer.weight, new_embeddings.weight])[token_ids]
    else:
        token_embeddings = embedding_layer(token_ids)

    pos_embeddings = positional_embedding(positions) * 100


    return list(zip(token_embeddings[0], pos_embeddings[0]))

def cosine_distance(emb1, emb2):
    return 1 - torch.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0)).item()

def token_distance(token1, token2, w_emb=0.5, w_pos=0.5, alpha=2, beta=0.1):
    emb1, pos1 = token1
    emb2, pos2 = token2

    emb_dist = cosine_distance(emb1, emb2) ** alpha

    pos_dist = math.tanh(beta * torch.abs(pos1 - pos2).float().mean().item())

    distance = w_emb * emb_dist + w_pos * pos_dist

    return distance

def n_gram_similarity(ref_tokens, pred_tokens, n, max_d=2.0):
    ref_ngrams = [ref_tokens[i:i+n] for i in range(len(ref_tokens)-n+1)]
    pred_ngrams = [pred_tokens[i:i+n] for i in range(len(pred_tokens)-n+1)]

    L_n = min(len(ref_ngrams), len(pred_ngrams))
    if L_n == 0:
        return 0

    total_distance = sum(
        sum(token_distance(ref_token, pred_token)
            for ref_token, pred_token in zip(ref_ngram, pred_ngram))
        for ref_ngram, pred_ngram in zip(ref_ngrams[:L_n], pred_ngrams[:L_n])
    )

    return 1 - (total_distance / (L_n * n)) #1 - (total_distance / (L_n * n * max_d))

def texbleu(reference, prediction, max_n=2, weights=None):
    if weights is None:
        weights = [1/max_n] * max_n

    ref_tokens = get_token_embeddings(reference)
    pred_tokens = get_token_embeddings(prediction)

    n_gram_scores = [n_gram_similarity(ref_tokens, pred_tokens, n)
                     for n in range(1, max_n+1)]

    ref_length = len(ref_tokens)
    pred_length = len(pred_tokens)

    if pred_length > ref_length:
        bp = 1
    else:
        bp = math.exp(1 - ref_length / pred_length)

    length_ratio = min(ref_length, pred_length) / max(ref_length, pred_length)

    bleu_score = math.exp(sum(w * math.log(max(s, 1e-10))
                      for w, s in zip(weights, n_gram_scores)))

    final_score = bleu_score

    return final_score


load_models_and_tokenizer()

In [ ]:
!ls /content

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
! ls -a


In [ ]:
import cv2
import os
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import random
import re
import json

from texteller import load_model, load_tokenizer, img2latex

json_save_file = "/content/results.json"
recognized_save_file = '/content/recognized.txt'

model = load_model(use_onnx=False)
tokenizer = load_tokenizer()

max_total_size = 10000
max_subset_size = 1000
ground_truth_index = {}

path = '/content/PRINTED_TEX_230k'
images_folder = f'{path}/generated_png_images'
formulas_listing_file = f'{path}/corresponding_png_images.txt'
ground_truth_file = f'{path}/final_png_formulas.txt'

pairs = []

with open(formulas_listing_file, 'r', encoding='utf-8') as f_links, \
     open(ground_truth_file, 'r', encoding='utf-8') as f_formulas:

    for link, formula in zip(f_links, f_formulas):
        if len(pairs) >= max_total_size:
            break
        pairs.append((link.strip(), formula.strip()))

random.shuffle(pairs)
iteration = 0

pairs = pairs[:max_subset_size]
ground_truth_index = {link: formula for link, formula in pairs}

predictions_vidal = []
predictions_BLEU = []
predictions_TeXBLEU = []

json_results = []

print('=====Model Work=====')

with open(recognized_save_file, 'w', encoding='utf-8') as save_file:
    iter = 0
    for filename in os.listdir(images_folder):

        if filename not in ground_truth_index:
            continue
        if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        img_path = os.path.join(images_folder, filename)
        img = Image.open(img_path)
        if img is None:
            print(f"Не удалось открыть {img_path}")
            continue

        print(f'{iter}/{max_subset_size}: {filename}')

        recognized_raw = img2latex(model, tokenizer, [img_path])
        #print(recognized_raw)
        recognized = recognized_raw[0]
        recognized_one_line = re.sub(r"\s+", " ", recognized).strip()

        save_file.write(recognized_one_line + "\n")

        gt = ground_truth_index[filename]

        vidal = marzal_vidal(gt, recognized_one_line)
        bleu = bleu_score_str(gt, recognized_one_line)
        texb = texbleu(gt, recognized_one_line)

        print(gt)
        print(recognized_one_line)
        print(f'vidal accuracy: {vidal}')
        print(f'BLEU score: {bleu}')
        print(f'TeXBLEU score: {texb}')

        predictions_vidal.append(vidal)
        predictions_BLEU.append(bleu)
        predictions_TeXBLEU.append(texb)

        json_results.append({
            "filename": filename,
            "ground_truth": gt,
            "recognized": recognized,
            "marzal_vidal_accuracy": vidal,
            "BLEU_accuracy": bleu,
            "TeXBLEU_accuracy": texb
        })

        iter += 1


with open('average.txt', 'w', encoding='utf-8') as f:
    f.write(f'Vidal average: {sum(predictions_vidal) / len(predictions_vidal)}\n')
    f.write(f'BLEU average: {sum(predictions_BLEU) / len(predictions_BLEU)}\n')
    f.write(f'TeXBLEU average: {sum(predictions_TeXBLEU) / len(predictions_TeXBLEU)}\n')

print('Vidal average:', sum(predictions_vidal) / len(predictions_vidal))
print('BLEU average:', sum(predictions_BLEU) / len(predictions_BLEU))
print('TeXBLEU average:', sum(predictions_TeXBLEU) / len(predictions_TeXBLEU))


# ---- гистограммы ----
plt.hist(predictions_vidal, 100, color='skyblue', edgecolor='black')
plt.xlabel('Значение')
plt.ylabel('Частота')
plt.title('Распределение значений по Marzal-Vidal')
plt.savefig("TexTeller_latex_vidal.png")
plt.clf()

plt.hist(predictions_BLEU, 100, color='skyblue', edgecolor='black')
plt.xlabel('Значение')
plt.ylabel('Частота')
plt.title('Распределение значений по BLEU')
plt.savefig("TexTeller_latex_BLEU.png")
plt.clf()

plt.hist(predictions_TeXBLEU, 100, color='skyblue', edgecolor='black')
plt.xlabel('Значение')
plt.ylabel('Частота')
plt.title('Распределение значений по TEXBLEU')
plt.savefig("TexTeller_latex_TeXBLEU.png")
plt.clf()

print('rdy')
